In [1]:
# %%capture
# If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed

!pip install --no-index /kaggle/input/packages/wheelhouse/bitsandbytes-0.43.0-py3-none-manylinux_2_24_x86_64.whl --find-links=/kaggle/input/packages/wheelhouse/
!pip install --no-index /kaggle/input/packages/wheelhouse/accelerate-0.28.0-py3-none-any.whl --find-links=/kaggle/input/packages/wheelhouse/
!pip install --no-index /kaggle/input/packages/wheelhouse/transformers-4.39.1-py3-none-any.whl --find-links=/kaggle/input/packages/wheelhouse/
!pip install --no-index /kaggle/input/packages/wheelhouse/optimum-1.17.1-py3-none-any.whl --find-links=/kaggle/input/packages/wheelhouse/
!pip install --no-index /kaggle/input/packages/wheelhouse/peft-0.10.0-py3-none-any.whl --find-links=/kaggle/input/packages/wheelhouse/
!pip install --no-index /kaggle/input/packages/wheelhouse/trl-0.8.1-py3-none-any.whl --find-links=/kaggle/input/packages/wheelhouse/
!pip install --no-index /kaggle/input/packages/wheelhouse/datasets-2.18.0-py3-none-any.whl --find-links=/kaggle/input/packages/wheelhouse/

Looking in links: /kaggle/input/packages/wheelhouse/
Processing /kaggle/input/packages/wheelhouse/bitsandbytes-0.43.0-py3-none-manylinux_2_24_x86_64.whl
Looking in links: /kaggle/input/packages/wheelhouse/
Processing /kaggle/input/packages/wheelhouse/accelerate-0.28.0-py3-none-any.whl
accelerate is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Looking in links: /kaggle/input/packages/wheelhouse/
Processing /kaggle/input/packages/wheelhouse/transformers-4.39.1-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
Looking in links: /kaggle/input/packages/wheelhouse/
Processing /kaggle/input/packages/wheelhouse/optimum-1.17.1-py3-none-any.whl
Processing /kaggle/input/packages/wheelhouse/coloredlogs-15.0.1-py2.py3-none-any.whl (from optimum==1.17.1)
Processing /kaggle/input

In [2]:
!wandb disabled

W&B disabled.


In [3]:
from accelerate.utils import BnbQuantizationConfig
from accelerate import Accelerator
import transformers
from transformers import TrainingArguments
import optimum
import bitsandbytes
from peft import prepare_model_for_kbit_training, LoftQConfig, LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
#from trl import SFTTrainer
#from datasets import Dataset
import datetime
start_time = datetime.datetime.now()
import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig
from peft import PeftModel
from tqdm import tqdm
import torch
from accelerate import Accelerator

accelerator = Accelerator()

# Comment/Uncomment and use as per wish

# MODEL_PATH = "/kaggle/input/gemma/transformers/7b-it/2"
# MODEL_PATH = "/kaggle/input/gemma/transformers/2b-it/2"

MODEL_PATH = "/kaggle/input/gemma_finetuned/transformers/v1/1/finetuned_model"

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config = quantization_config,
)

model = PeftModel.from_pretrained(model, MODEL_PATH)
model = accelerator.prepare(model)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
import pandas as pd
from tqdm import tqdm

def truncate_txt(text, length):
    text_list = text.split()
    
    if len(text_list) <= length:
        return text
    
    return " ".join(text_list[:length])


def gen_prompt(og_text, rewritten_text):
    
    og_text = truncate_txt(og_text, 200)
    rewritten_text = truncate_txt(rewritten_text, 200)
    
    return (f"Given below are 2 texts, the Rewritten text was created from the Original text using the google Gemma model. You are trying to understand how the original text was transformed into a new version. Analyze the changes in style and theme and come up with a prompt that must have been used to transform the Original Text to the Rewritten Text. Start your output with the string \"Prompt:\". Also limit your output to just the rewrite prompt. Do not include the original text and the rewritten text in your response.\n"
            f"""Original Text:\"""{og_text}\"""\nRewritten Text:\"""{rewritten_text}\"""\n""")

In [6]:
#TEST_DF_FILE = '/kaggle/input/gemma-rewrite-nbroad/nbroad-v2.csv'

TEST_DF_FILE = '/kaggle/input/llm-prompt-recovery/test.csv'
SUB_DF_FILE = '/kaggle/input/llm-prompt-recovery/sample_submission.csv'

tdf = pd.read_csv(TEST_DF_FILE,usecols=['id', 'original_text', 'rewritten_text'])
sub = pd.read_csv(SUB_DF_FILE,usecols=['id', 'rewrite_prompt'])

In [7]:
import gc
import re

device = accelerator.device
tdf['id'] = sub['id'].copy()

pbar = tqdm(total=tdf.shape[0])

it = iter(tdf.iterrows())
idx, row = next(it, (None, None))

DEFAULT_TEXT = "Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style."

res = []

while idx is not None:
    
    if (datetime.datetime.now() - start_time) > datetime.timedelta(hours=8, minutes=30):
        res.append([row["id"], DEFAULT_TEXT])
        idx, row = next(it, (None, None))
        pbar.update(1)
        continue
        
    torch.cuda.empty_cache()
    gc.collect()
        
    try:        
        messages = [
            {
                "role": "user",
                "content": gen_prompt(row["original_text"], row["rewritten_text"])
            }
        ]
        encoded_input = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(device)
        
        with torch.no_grad():
            encoded_output = model.generate(encoded_input, max_new_tokens=100, do_sample=True, pad_token_id=tokenizer.eos_token_id)
        
        decoded_output = tokenizer.batch_decode(encoded_output, skip_special_tokens=True)[0]
        
        decoded_output = re.sub(r"[\s\S]*model", 'model', decoded_output, 1)

        # Updated regex to capture everything after 'model' leading up to the quoted prompt
        match = re.search(r'Prompt:s*(.+)', decoded_output, re.DOTALL)
        if match is not None:
            prompt = match.group(1)
            #print(prompt)
            res.append([row["id"], prompt])
        else:
            res.append([row["id"], DEFAULT_TEXT])
                            
    except Exception as e:
        print(f"ERROR: {e}")
        res.append([row["id"], DEFAULT_TEXT])
        
    finally:
        idx, row = next(it, (None, None))
        pbar.update(1)

        
pbar.close()

  0%|          | 0/1 [00:00<?, ?it/s]2024-03-26 23:28:51.850766: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 23:28:51.850895: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 23:28:51.976484: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|██████████| 1/1 [00:19<00:00, 20.00s/it]


In [8]:
sub = pd.DataFrame(res, columns=['id', 'rewrite_prompt'])
sub.to_csv("submission.csv", index=False)

In [9]:
sub['rewrite_prompt']

0    \n\n**Make the text sound more creative, whims...
Name: rewrite_prompt, dtype: object

In [10]:
res

[[9559194,
  '\n\n**Make the text sound more creative, whimsical, and engaging, as if rewritten by a skillful storyteller.**\n\n**Add a conversational, humorous tone.**\n\n**Introduce a catchy chorus that summarizes the goal of the competition.**\n\n**Transform the original text into a shanty-like poem.**']]